In [2]:
import cv2
import numpy as np
import pandas as pd
import os
import scipy.stats as sp
import glob
import time
from datetime import datetime

In [3]:
# 구글 드라이브랑 연동시키는 코드
# 데이터 불러오기 위해서 필요
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Change name of folder

In [ ]:
# cycle_start = 1
# cycle_end = 4

case_list = [2,3,5,6]

# start = 1
# end = 15

# rep = 6
# py = "D:/Monitoring/Main/Image/{0}/pyrometer".format(c)


ccd = "/content/drive/Shareddrives/S-hero 디독스팀/ccd/" #이미지 데이터 경로
py = "/content/drive/Shareddrives/S-hero 디독스팀/pyrometer/" #고온계 데이터 경로

fl = os.listdir(ccd)
print(fl)
# for case in case_list:
#     os.rename(ccd+"{0}" .format(case), ccd+"Case{0}" .format(case)) #이미지 데이터 폴더명 변경(이미 맞게 변경 되어있음)
#     os.rename(py+"{0}.dat" .format(case), py+"Case{0}.dat" .format(case)) #고온계 dat파일명 변경


['Case2', 'Case3', 'Case5', 'Case6']


## Image data feature extraction

In [ ]:
# 1사이클 내 start case부터 end case까지 이미지 처리함
# start = 1
# end = 15


# 이미지 처리할 사이클 범위 지정
cycle_start = 36
cycle_end = 36

case_list = [2,3,5,6]

for cycle in range(cycle_start, cycle_end+1):
    directory = "/content/drive/Shareddrives/S-hero 디독스팀/ccd" #이미지 데이터 경로
    save_dir = "/content/drive/Shareddrives/S-hero 디독스팀/extracted_feature"  #추출된 특징값 행렬 저장 경로
    
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
    
    for case in case_list:
        initial_dir = directory + '/Case{0}/'.format(case)

      
        #폴더 생성 *******************************************
        dir_name = initial_dir + "/Thresholding" 
        if not os.path.isdir(dir_name):
            os.mkdir(dir_name)
        # ****************************************************


    #     파일명 변경 *******************************************
        # files = os.listdir(initial_dir)
        # files = sorted(files, key=lambda k: (len(k.split('-')[-1].split('.')[0]))) #각 이미지 파일들 하이픈 뒤의 마지막 숫자 오름차순 정렬

        # t=0
        # for f in files:
        #     t+=1
        #     if "jpg" in f:
        #         os.rename(initial_dir+f,initial_dir+"Case{0} ({1}).jpg".format(case,t))
        #     if t%100 == 0:
        #         print("{0} - Case {1} : Renaming....{2}/{3}" .format(cycle, case, t,len(files)))

        # print('************** Renaming  End  **************')
    #     ******************************************************

        fps = 30 # CCD sampling frequency
        start_time = time.time() # 코드 시작 시간

        #파일경로에 있는 이미지 이름 리스트 생성
        path = initial_dir
        save_path = dir_name
        img_path = path+'/*.jpg'
        image_name = glob.glob(img_path)
        print("Case {0} : num of image: {1}" .format(case, len(image_name)))

        valid_image_number = []

        # 번호대로 순서 정렬
        image_name = sorted(image_name, key=lambda f: int(f.split(' ')[-1].split('.')[0][1:-1]))
        num_image = len(image_name)

        # 추출할 특징값 리스트
        feature_name = ['Area', 'Aspect ratio', 'Width', 'Height', "Center_X", "Center_Y"
                        , 'E_aspect ratio', 'Ellipse_1', "Ellipse_2", 'Angle', 'Mean_val'
                        ,'Max_val', 'Intensity', 'Extent', 'Solidity']
        
        # 특징값 저장될 배열 생성
        feature = np.zeros([num_image,len(feature_name)])

        i=0
        for fname in image_name:
            i+=1

            if i%100 == 0:
                print("{0} - Case {1} : Mask Making....{2}/{3}" .format(cycle, case, i,len(image_name)))

            if i == len(image_name):
                print('**************  End  **************n')

            img = cv2.imread(fname)

            # 이미지 반시계방향 90도 회전
            h,w,_ = img.shape
            matrix = cv2.getRotationMatrix2D((w/2, h/2), 90, 1)
            img = cv2.warpAffine(img, matrix, (w, h))

            img_g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Gray 이미지
            ret, img_t = cv2.threshold(img_g, 120, 255, cv2.THRESH_BINARY) # Thresholding
            img_c = cv2.cvtColor(img_g, cv2.COLOR_GRAY2RGB) # RGB 이미지    
            img_mask = cv2.bitwise_and(img_g, img_g, mask=img_t) # 마스크 이미지

            contours, hierarchy = cv2.findContours(img_mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

            # 이미지 내 컨투어가 하나 뿐일 때
            if len(contours) != 1:
                contours = np.array(contours)
            cnt = np.zeros_like(contours)
            area = np.zeros_like(contours)

            # 이미지 내 컨투어가 다수 존재할 때 
            for j in range(len(contours)):
                cnt[j] = len(contours[j])
                area[j] = cv2.contourArea(contours[j])

            #컨투어가 있을 경우, 가장 큰 컨투어 찾기
            try: 
                if len(contours) == 1:
                    contour = contours
                else:
                    max_area = max(area) #가장 큰 면적의 컨투어
                    ind = np.where(area == max_area)
                    contour = contours[ind]  # 이미지의 컨투어

                # 마스크 통해 ROI 생성
                zero = np.zeros_like(img_g)
                cv2.drawContours(zero, contour, 0, (255), -1)
                roi = cv2.bitwise_and(img_g, img_g, mask=zero)

                #특징값 추출
                area = cv2.contourArea(contour[0]) # Area : 용융풀 면적(픽셀 개수)
                x,y,width,height = cv2.boundingRect(contour[0]) # Width / Height : 용융풀 너비, 높이
                aspect_ratio = float(width)/height # Aspect ratio : 용융풀 종횡비
                moment = cv2.moments(contour[0])
                center_x = int(moment['m10']/moment['m00']) # Center X : 용융풀 X좌표
                center_y = img.shape[0] - int(moment['m01']/moment['m00']) # Center Y : 용융풀 Y좌표
                
                # 용융풀 ROI를 타원에 매핑 --> Ellipse_1(장축), Ellipse_2(단축), Angle(타원 장축-단축 각도)
                (x,y),(ellipse_1,ellipse_2),angle = cv2.fitEllipse(contour[0])
                e_aspect_ratio = float(ellipse_2)/ellipse_1 # E_aspect ratio : 용융풀 ROI 타원 종횡비
                mean_val = cv2.mean(img_g, mask=zero)[0] # Mean_val : 용융풀의 grayscale 평균
                max_val = np.max(img_g) # Max_val : 용융풀 grayscale의 최대값
                intensity = mean_val * area # Intensity : 용융풀 총 grayscale (mean_val * area)
                extent = area/(width*height) # Extent : 용융풀 bounding rect에 대한 용융풀 면적의 비

                hull = cv2.convexHull(contour[0]) 
                solidity = area/cv2.contourArea(hull) # Solidity : 용융풀 Convexhull에 대한 용융풀 면적 비

                #특징값 저장
                feature[i-1,:] = [area, aspect_ratio, width, height, center_x, center_y
                                  , e_aspect_ratio, ellipse_1, ellipse_2, angle, mean_val
                                  , max_val, intensity, extent, solidity]

            # ROI 이미지 저장
                save_name = save_path+'/ROI ({0}).jpg' .format(i)
                cv2.imwrite(save_name, roi)

            except:  #컨투어가 없을 경우 특징값 0으로 저장
                feature[i-1,:] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

                save_name = save_path+'/ROI ({0}).jpg' .format(i)
                cv2.imwrite(save_name, img)

        print("time : ", time.time()-start_time, '(s)\n')

        # 특징값 csv파일로 저장
        result_name = save_dir + '/{0}-Case{1}.csv'.format(cycle, case)
        result = pd.DataFrame(feature, index=np.arange(0, feature.shape[0])/fps,columns=feature_name)
        result.to_csv(result_name, header=feature_name)

        
now = datetime.now()
print('\n\n>>>> {0}' .format(now.isoformat().split('T')[0]))
print('>>>> {0}' .format(now.isoformat().split('T')[1]))

Case 2 : num of image: 1017


## Pyrometer data feature extraction

In [ ]:
# c = 15
# start=1
# end=15

case_list = [2,3,5,6]

cycle_start = 36
cycle_end = 36

# 데이터 이름 정의
for cycle in np.arange(cycle_start, cycle_end+1):
    data_directory_py = "/content/drive/Shareddrives/S-hero 디독스팀/pyrometer/"  #dat 저장된 경로
    csv_save_directory = "/content/drive/Shareddrives/S-hero 디독스팀/pyrometer/processed/" #csv 저장할 경로
    save_directory = '/content/drive/Shareddrives/S-hero 디독스팀/extracted_feature/pyrometer/' #처리된 데이터 저장할 경로
    
#     processed_dir_py = save_directory + '/processed'

    # 폴더가 없으면 폴더 생성
    if not os.path.isdir(csv_save_directory):
        os.mkdir(csv_save_directory)
    if not os.path.isdir(save_directory):
        os.mkdir(save_directory)

    for case in case_list:
        print("{0}-Case{1} : Processing..." .format(cycle,case))    
        f_name_py = data_directory_py + "Case{0}.dat" .format(case)
#         f_name_py = data_directory_py + "case{0}.dat" .format(case)

        #데이터 불러오기
        f_py = open(f_name_py, 'r',encoding = 'unicode_escape')

        # 앞 뒤 열 제거 (이상한 정보 빼기)
        f_py = pd.DataFrame(f_py)[9:-2][0]

        # 데이터 나누기 
        f_split = pd.DataFrame(f_py.str.split('\t').tolist())

        # 시간, 온도 데이터만 select
        data_py = pd.DataFrame({'Time' : f_split[0], 'Temp' : f_split[1]})

        # csv파일로 저장
        data_py.to_csv(csv_save_directory+'Case{0}.csv'.format(case))

        # 레이저 꺼진 데이터 빼기
        beg_py = 0
        end_py = 0
        row_num_py = data_py.shape[0] #데이터 개수
        layers_py = [] # track별 데이터
        num_py = [] # #track별 데이터 개수
        thres_py = 490

#         successive_data_py = np.zeros((1,5))
        
        #레이저 꺼진 데이터 빼기
        for i in np.arange(row_num_py-1):
#             successive_data_py[:] = data_py.iloc[i:i+, 1].values.reshape(1,5)
#             if np.float(data_py.iloc[i,1])<=800 and np.float(data_py.iloc[i+1,1])<=800: #연속된 두 데이터가 threshold보다 작을 때
            if np.float(data_py.iloc[i,1])==thres_py and np.float(data_py.iloc[i+1,1])==thres_py:
#             if np.all(successive_data_py == thres_py):
                beg_py = i+1

#             if np.float(data_py.iloc[i,1])>800 and np.float(data_py.iloc[i+1,1])<=800:
            if np.float(data_py.iloc[i,1])!=thres_py and np.float(data_py.iloc[i+1,1])==thres_py:
#             if successive_data_py[0][0] != thres_py and np.all(successive_data_py[0][1:] == thres_py):
                end_py = i
                layers_py.append((beg_py+4, end_py-4))
                num_py.append(end_py-4-beg_py-4)

        layers_py = layers_py[3:-2]
        num_py = num_py[3:-2]
        processed_path_py = save_directory + '/{0}-Case{1}.csv'.format(cycle,case)
        sampling_f = 10

        new_data_py = np.zeros((0,1))


        for layer in layers_py:
            layer_data = data_py.iloc[layer[0]:layer[1], 1].values.reshape(-1,1)
            new_data_py = np.concatenate((new_data_py, layer_data), axis=0)


#         try:
        new_data_py = pd.DataFrame(new_data_py, index=np.arange(0, new_data_py.shape[0])/sampling_f, columns=data_py.columns[1:] )
        new_data_py.to_csv(processed_path_py)

#         except:
#             new_data_py = pd.DataFrame(new_data_py, index=np.arange(0, new_data_py.shape[0]/sampling_f-0.5/sampling_f, 1/sampling_f), columns=data_py.columns[1:] )
#             new_data_py.to_csv(processed_path_py)

now = datetime.now()
print('\n\n>>>> {0}' .format(now.isoformat().split('T')[0]))
print('>>>> {0}' .format(now.isoformat().split('T')[1]))

36-Case2 : Processing...
36-Case3 : Processing...
36-Case5 : Processing...
36-Case6 : Processing...


>>>> 2021-09-28
>>>> 14:29:45.524265


## Data preprocessing _ 수정필요

In [4]:
cycle_start = 36
cycle_end = 36
start = 1
end = 15

case_list = [2,3,5,6]

for cycle in np.arange(cycle_start, cycle_end+1):
    directory = '/content/drive/Shareddrives/S-hero 디독스팀/extracted_feature/' ############### 저장된 피쳐 경로
    
    num_of_layers = [] # 공정변수별 트랙 개수 저장할 배열

    processed_dir = directory + '/processed' # 전처리 데이터 저장할 폴더 경로

    if not os.path.isdir(processed_dir):
        os.mkdir(processed_dir)

    for case in case_list:
        print("{0}-Case{1} : Processing..." .format(cycle,case))

        path = directory + '/{0}-Case{1}.csv'.format(cycle,case)

        data = pd.read_csv(path) # 특징값 데이터 불러오기

        row_num = data.shape[0] 
        col_num = data.shape[1]


        track_beg = 0
        track_end = 0
        layers = [(0,0)]
        num = []

        successive_data = np.zeros((1,5))
        
        ## 레이저 꺼진 데이터 빼기
        for i in np.arange(row_num-4):
            successive_data[:] = data.iloc[i:i+5]['Area'].values.reshape(1,5)
            if np.all(successive_data == 0): #아직 안생김, track_beg을 5칸 뒤로 이동 =>시작점 찾기
                track_beg = i+5

            if successive_data[0][0] != 0 and np.all(successive_data[0][1:] == 0): #첫번째 넓이 0이 아닌데 그 뒤의 나머지는 0 => 맨 끝 도달
                track_end = i
                
                
                if track_end!=track_beg and track_beg!=(layers[-1][0]-13): #1 개만 스파크튀어서 값이 떳을때 제외
                    layers.append((track_beg+13, track_end-11)) # 가감속 구간 영향 제외하려고 앞 뒤로 데이터 지우기
                    num.append(track_end - 13 - track_beg - 11) # layer당 데이터 갯수 count
        
        # 앞에 3개의 데이터 버리기(첫번째 데이터는 레이저 안정화 트랙, 두세번째 데이터는 에열되는 트랙의 데이터)
        # 뒤에 2개의 데이터 버리기(애칭된 절단면 노이즈 때문)
        # 데이터의 시작과 끝의 인덱싱을 모아둔 리스트
        laser_start_index = layers[1][0] #레이저가 제일 처음나올때 데이터 인덱스
        ref_Center_Y = data.iloc[laser_start_index, 6]

        layers = layers[4:-2] #제일 처음 (0,0) 추가해서 한 개 더빼기
        num = num[3:-2]
        num_of_layers.append(len(num))

        processed_path = processed_dir + '/{0}-processed_case{1}.csv'.format(cycle,case)

        fps = 30

        # Center_Y 상대값
        data.iloc[:,6] = data.iloc[:,6] - ref_Center_Y
        
        new_data = np.zeros((0,col_num-1))

        for layer in layers:
            new_data = np.concatenate((new_data, data.iloc[layer[0]:layer[1], 1:]), axis=0)


        new_data = pd.DataFrame(new_data, index=np.arange(0, new_data.shape[0])/fps, columns=data.columns[1:] )
        new_data.to_csv(processed_path)

    num_of_layers = pd.DataFrame(num_of_layers)
    print("Num of layers: ",num_of_layers)
    num_of_layers.to_csv(processed_dir+"/num_of_layers.csv")
    
now = datetime.now()
print('\n\n>>>> {0}' .format(now.isoformat().split('T')[0]))
print('>>>> {0}' .format(now.isoformat().split('T')[1]))

36-Case2 : Processing...


IndexError: ignored

In [ ]:
# cycle_start = 36
# cycle_end = 36
# start = 1
# end = 15

# case_list = [2,3,5,6]

# for cycle in np.arange(cycle_start, cycle_end+1):
#     directory = '/content/drive/Shareddrives/S-hero 디독스팀/extracted_feature/' ############### 저장된 피쳐 경로
    
#     num_of_layers = [] # 공정변수별 트랙 개수 저장할 배열

#     processed_dir = directory + '/processed' # 전처리 데이터 저장할 폴더 경로

#     for case in case_list:
#         print("{0}-Case{1} : Processing..." .format(cycle,case))

#         path = directory + '/{0}-Case{1}.csv'.format(cycle,case)

#         data = pd.read_csv(path) # 특징값 데이터 불러오기

#         row_num = data.shape[0] 
#         col_num = data.shape[1]
#         print("case{} shape:".format(case),data.shape)


#         track_beg = 0
#         track_end = 0
#         layers = [(0,0)]
#         num = []

#         successive_data = np.zeros((1,5))
        
#         ## 레이저 꺼진 데이터 빼기
#         for i in np.arange(row_num-4):
#             successive_data[:] = data.iloc[i:i+5]['Area'].values.reshape(1,5)
#             if np.all(successive_data == 0):
#                 track_beg = i+5

#             if successive_data[0][0] != 0 and np.all(successive_data[0][1:] == 0):
#                 track_end = i
                
                
#                 if track_end!=track_beg and track_beg!=(layers[-1][0]-13): #1 개만 스파크튀어서 값이 떳을때 제외
#                     layers.append((track_beg+13, track_end-11)) # 가감속 구간 영향 제외하려고 앞 뒤로 데이터 지우기
#                     num.append(track_end - 13 - track_beg - 11) # layer당 데이터 갯수 count
        
#         # 앞에 3개의 데이터 버리기(첫번째 데이터는 레이저 안정화 트랙, 두세번째 데이터는 에열되는 트랙의 데이터)
#         # 뒤에 2개의 데이터 버리기(애칭된 절단면 노이즈 때문)
#         # 데이터의 시작과 끝의 인덱싱을 모아둔 리스트
#         laser_start_index = layers[1][0] #레이저가 제일 처음나올때 데이터 인덱스
#         ref_Center_Y = data.iloc[laser_start_index, 6]

#         layers = layers[4:-2] #제일 처음 (0,0) 추가해서 한 개 더빼기
#         num = num[3:-2]
#         num_of_layers.append(len(num))

#         processed_path = processed_dir + '/{0}-processed_case{1}.csv'.format(cycle,case)

#         fps = 30

#         # Center_Y 상대값
#         data.iloc[:,6] = data.iloc[:,6] - ref_Center_Y
        
#         new_data = np.zeros((0,col_num-1))

#         for layer in layers:
#             new_data = np.concatenate((new_data, data.iloc[layer[0]:layer[1], 1:]), axis=0)


#         new_data = pd.DataFrame(new_data, index=np.arange(0, new_data.shape[0])/fps, columns=data.columns[1:] )
#         new_data.to_csv(processed_path)

#     num_of_layers = pd.DataFrame(num_of_layers)
#     print("Num of layers: ",num_of_layers)
#     num_of_layers.to_csv(processed_dir+"/num_of_layers.csv")
    
# now = datetime.now()
# print('\n\n>>>> {0}' .format(now.isoformat().split('T')[0]))
# print('>>>> {0}' .format(now.isoformat().split('T')[1]))

36-Case2 : Processing...
case2 shape:  (4249, 3)
36-Case3 : Processing...
case3 shape:  (3150, 3)
36-Case5 : Processing...
case5 shape:  (2240, 3)
36-Case6 : Processing...
case6 shape:  (1724, 3)
Num of layers:  Empty DataFrame
Columns: []
Index: []


FileNotFoundError: ignored